<a href="https://colab.research.google.com/github/anilbhatt1/EVA4_Phase1/blob/master/EVA4_Session_2_With_explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28x28x1  Output - 28x28x32 : RF -> 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # i/p - 28x28x32 o/p - 28x28x64 : RF -> 5
        self.pool1 = nn.MaxPool2d(2, 2) # i/p - 28x28x64 o/p 14x14x64 : RF -> 6
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # i/p 14x14x64 o/p 14x14x128 : RF -> 10
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # i/p 14x14x128 o/p 14x14x256 : RF -> 14
        self.pool2 = nn.MaxPool2d(2, 2)  # i/p 14x14x256 o/p 7x7x256   : RF -> 16
        self.conv5 = nn.Conv2d(256, 512, 3)  # i/p 7x7x256 o/p 5x5x512 -> RF 24
        self.conv6 = nn.Conv2d(512, 10, 5) # i/p 5x5x512 o/p 1x1x10 -> RF 40
#        self.conv7 = nn.Conv2d(1024, 10, 3)  # i/p 3x3x1024 o/p 1x1x10 -> RF 40

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#       x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv5(x))
#        x = F.relu(self.conv7(x))
#        x = self.conv7(x)
        x = self.conv6(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8             [-1, 10, 1, 1]         128,010
Total params: 1,696,010
Trainable params: 1,696,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.44
Params size (MB): 6.47
Estimated Total Size (MB): 7.91
----------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)  # This is to generate random numbers
batch_size = 128

# num_workers -> which denotes the number of processes that generate batches in parallel. A high enough number of workers assures that 
# CPU computations are efficiently managed. 0 means data will be loaded in the main process.
# pin_memory -> If you load your samples in the Dataset on CPU and would like to push it during training to the GPU, you can speed up 
# the host to device transfer by enabling pin_memory . This lets your DataLoader allocate the samples in page-locked memory, which 
# speeds-up the transfer. 

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),                        # Will transform (H,W,C) to (C,W,H) and scale between 0-1
                        transforms.Normalize((0.1307,), (0.3081,))    # 0.1307 - MNIST mean, 0.3081 - MNIST std
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


0it [00:00, ?it/s]

9920512it [00:05, 1817961.12it/s]                             


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


32768it [00:00, 290612.85it/s]                           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:00, 4850306.36it/s]                           
8192it [00:00, 117971.33it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm  

# Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done!

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates 
        # the gradients on subsequent backward passes
        output_train = model(data)
        loss = F.nll_loss(output_train, target)  # Negative log likehood. Loss calculated between prediction from model & input target
        loss.backward()  # Operation that calculates gradient (back-prop)
        optimizer.step() # Operation that updates the weights (back-prop)
        pbar.set_description(desc= f'ding loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():  # We are disabling gradient calculation as we are dealing with test data
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output_test = model(data)
            test_loss += F.nll_loss(output_test, target, reduction='sum').item()  # sum up batch loss
            pred = output_test.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            # .eq -> Computes element-wise equality between values in pred and target. 
            # To enable comparison we are changing target to same shape as pred using target.view_as(pred)
            # Output of pred.eq(target.view_as(pred)) will be 1 (True) or 0(False)
            # We will sum-up this with 'correct' for each of test row
            # item() will help to convert output of pred.eq(target.view_as(pred)).sum() to a proper number
            # Basically we are trying to get count of correctly predicted records from test dataset
            # We will use this 'correct' to predict accuracy i.e. accuracy = correct/len(test_loader.dataset)

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

ding loss=0.1440131813287735 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 29.14it/s]



Test set: Average loss: 0.0456, Accuracy: 9858/10000 (99%)

